# Tutorial IPOPT

Example problem: number 71 from the Hock-Schittkowsky test suite.

$min_{x_1, x_2, x_3, x_4}\quad x_1 x_4 (x_1 + x_2 + x_3) + x_3$

$x_1^2 + x_2^2 + x_3^2 + x_4^2 = 40$

$1 \leq x_i \leq 5$

Initial condition:

$\mathbf{x}_0 = (1, 5, 5, 1)$

Optimal solution:

$\mathbf{x}^{\star} = (1.0, 4.743, 3.821, 1.379)$

## Import statement

In [17]:
import ipopt
import scipy.sparse as sps
import numpy as np

## Implementation

In [18]:
class hs071(object):
    def __init__(self):
        pass

    def objective(self, x):
        #
        # The callback for calculating the objective
        #
        return x[0] * x[3] * np.sum(x[0:3]) + x[2]

    def gradient(self, x):
        #
        # The callback for calculating the gradient
        #
        return np.array([
                    x[0] * x[3] + x[3] * np.sum(x[0:3]),
                    x[0] * x[3],
                    x[0] * x[3] + 1.0,
                    x[0] * np.sum(x[0:3])
                    ])

    def constraints(self, x):
        #
        # The callback for calculating the constraints
        #
        return np.array((np.prod(x), np.dot(x, x)))

    def jacobian(self, x):
        #
        # The callback for calculating the Jacobian
        #
        return np.concatenate((np.prod(x) / x, 2*x))

    def hessianstructure(self):
        #
        # The structure of the Hessian
        # Note:
        # The default hessian structure is of a lower triangular matrix. Therefore
        # this function is redundant. I include it as an example for structure
        # callback.
        #
        global hs

        hs = sps.coo_matrix(np.tril(np.ones((4, 4))))
        return (hs.col, hs.row)

    def hessian(self, x, lagrange, obj_factor):
        #
        # The callback for calculating the Hessian
        #
        H = obj_factor*np.array((
                (2*x[3], 0, 0, 0),
                (x[3],   0, 0, 0),
                (x[3],   0, 0, 0),
                (2*x[0]+x[1]+x[2], x[0], x[0], 0)))

        H += lagrange[0]*np.array((
                (0, 0, 0, 0),
                (x[2]*x[3], 0, 0, 0),
                (x[1]*x[3], x[0]*x[3], 0, 0),
                (x[1]*x[2], x[0]*x[2], x[0]*x[1], 0)))

        H += lagrange[1]*2*np.eye(4)

        #
        # Note:
        #
        #
        return H[hs.row, hs.col]

    def intermediate(
            self,
            alg_mod,
            iter_count,
            obj_value,
            inf_pr,
            inf_du,
            mu,
            d_norm,
            regularization_size,
            alpha_du,
            alpha_pr,
            ls_trials
            ):

        #
        # Example for the use of the intermediate callback.
        #
        print("Objective value at iteration #%d is - %g" % (iter_count, obj_value))

In [19]:
# Initial conditions
x0 = [1.0, 5.0, 5.0, 1.0]

# Lower bounds - variables
lb = [1.0, 1.0, 1.0, 1.0]

# Upper bounds - variables
ub = [5.0, 5.0, 5.0, 5.0]

# Lower bounds - constraints
cl = [25.0, 40.0]

# Upper bounds - constraints
cu = [2.0e19, 40.0]

nlp = ipopt.problem(
            n=len(x0),
            m=len(cl),
            problem_obj=hs071(),
            lb=lb,
            ub=ub,
            cl=cl,
            cu=cu
            )

# Setting optimization parameters
nlp.addOption('mu_strategy', 'adaptive')
nlp.addOption('tol', 1e-7)

# Optimization run
x, info = nlp.solve(x0)

# Solution
print(x)

Objective value at iteration #0 is - 16.1097
Objective value at iteration #1 is - 17.4104
Objective value at iteration #2 is - 18.0016
Objective value at iteration #3 is - 17.1995
Objective value at iteration #4 is - 16.941
Objective value at iteration #5 is - 17.0034
Objective value at iteration #6 is - 17.014
Objective value at iteration #7 is - 17.014
Objective value at iteration #8 is - 17.014
[1.         4.74299964 3.82114998 1.37940829]
